# Fitting Radio Lightcurves


## Key steps
- Plot the radio light curve
- Determine its spectral index
- Scale the data based on its spectral index
- Fit the data with a power law
- Fit the data with a broken power law with a smooth turnover


# Packages Import


In [1]:

import sys
sys.path = sys.path[:-2]
path_temp = sys.path
path = [sys.path[-1]]
path.extend(sys.path[:-1])
sys.path = path
import numpy as np
from astropy.io import ascii
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import matplotlib.cm as cmx
from scipy.optimize import least_squares, curve_fit
from scipy.stats import f
import emcee
import corner
import os
from timeit import default_timer as timer

# Step 1: Load the data
get the data out from papers into excel sheets, or any other forms that can be easily read by python

### This is the space to include data import

# Step 1: Make a plot of the radio lightcurve
We will now plot the flux density as a function of time (the lightcurve). We use different coloured markers to denote the observing frequency, and different marker styles to denote different telescopes.



# Step 2: Determining the spectral index
As with many things in astrophysics, the emission in the radio regime is _non-thermal_ in origin (unlike the early emission in the optical/UV/infrared, which is a thermal blackbody).  What that means is that generally has a power-law spectrum:
$$ S_\nu(\nu) \propto \nu^\alpha$$
Strictly this only works for data observed at _exactly_ the same time, but real observations don't work that way.  A single telescope can (usually) only observe at a single frequency, and different telescopes are separated in time by schedules, longitude, etc.  So we need to be a bit more generous about how we define simultaneous. Luckily, the light-curve is mostly evolving pretty slowly, so this isn't necessarily a problem.

Here are two functions to take a subset of the data that was observed roughly simultaneously and calculate the spectral index $\alpha$ and its uncertainty using multi-band observation at 162 days post-merger.

In [ ]:
def calc_power_law(freq,S0,alpha):
    S = S0 * (freq) ** alpha
    return S

def alpha_calc(data):    
    #Get lightcurve values
    freqs = data['frequency']
    flux = data['flux']
    flux_errs = data['rms']
    
    #Use the scipy curve_fit algorithm to calculate the best fit value
    popt, pcov = curve_fit(calc_power_law, freqs, flux ,sigma=flux_errs, p0=(50,-0.61),absolute_sigma=True)
    
    alpha = popt[1] #Best-fit spectral index
    alpha_err = np.sqrt(np.diag(pcov))[1] #Uncertainty in alpha
    
    return alpha, alpha_err

In [ ]:
#Select the data at the ~162 day epoch and print the spectral index + uncertainty

sel_data = data[data['delta_t'] == 162.89]

alpha, alpha_err = alpha_calc(sel_data)

print("alpha = %.1f+/-%.1f"%(alpha, alpha_err))

## Step 3: Scaling the data based on the spectral Index
Based on the $\alpha$ you calculated above, what happens if you assume that _all_ of the observations can be fit using the same frequency power-law?  i.e., if $\alpha$ were the same at all times?  If this is the case then scaling all of the data to a single frequency makes it easier to understand the temporal properties of the lightcurve as a function of only 1 variable, not 2.

Write a function to take the observed data and scale it to a specific frequency based on an estimated spectral index. 

Don't forget to include uncertainties! You should add two columns to your data table called "scaled_flux" and "scaled_rms".

Questions:
1. Can you think of reasons why the spectral index should stay the same?  Why it should change?

In [ ]:
def scale_data(data, alpha, alpha_err, ref_freq=3.0):
    #calculate a scaling factor for the flux density and uncertainty
    f_scale = ######
    rms_scale = ######
    
    #scale the flux and uncertainty - don't forget to add errors in quadrature
    scaled_flux = ######
    scaled_rms = ######
    
    #Add two new columns to the data
    data['scaled_flux'] = scaled_flux
    data['scaled_rms'] = scaled_rms
    
    return data

Scale the data to 3 GHz based on your estimated spectral index and associated uncertainty, then plot the scaled lightcurve by passing `scaled=True` to your `make_plot` function

In [ ]:
data = scale_data(data, alpha, alpha_err)
make_plot(data, scaled=True)

# Step 4: Fitting the data
We now want to characterise the radio lightcurve. You should be able to see that it initially rises according to a power law, peaks somewhere between 100 and 200 days post-merger and then declines according to a different power law.

Create a new table called tdata that we will use to determine the properties of the lightcurve

We can fit this data with a "smoothed broken power law", which combines two power laws with a smoothing parameter around the break point. One functional form of this is given by

$S_\nu(t) = S_{\rm \nu,peak} \left[ \left(\dfrac{t}{t_{\rm peak}}\right)^{-s\delta_1} + \left(\dfrac{t}{t_{\rm peak}}\right)^{-s\delta_2}\right]^{-1/s}$

Here the spectral index is still $\alpha$, but we've also introduced _temporal_ power-law indices $\delta_1$ (before the break) and $\delta_2$ (after the break).  $S_{\rm \nu,peak}$ is the flux density at peak, and $s$ controls the smoothness of the break.

Write a function smooth_broken_power_law() that outputs a smoothed broken power law that also scales based on frequency and spectral index



In [ ]:
def smooth_broken_power_law(t, nu, S_peak, t_peak, delta_1, delta_2, alpha, log_s, nu0=3.0):    
    S = ##### write your own code
    
    return S

## Using a Markov Chain Monte Carlo

We now want to fit a smoothed broken power law to our data. In our paper we do this via a parameter grid-search to minimise the goodness-of-fit parameter $\chi^2$, i.e., compute $\chi^2(S_{\nu,\rm peak},t_{\rm peak},\delta_1,\delta_2,\alpha,s)$ for a 6-dimension parameter grid. However, grid searches are slow and innefficient.  It's better to concentrate your effort in the part of the fit where the data "prefer" to go.  We can do this using a slightly more complicated statistical technique

Here we will perform an Markov Chain Monte Carlo (MCMC) fit using the [`emcee`](http://dfm.io/emcee/current/) package, to determine lightcurve parameters and the spectral index of the source. First you will need to write 3 functions that define your Probability, Prior and Likelihood.

We will use a uniform prior with $\delta_1>0$ (since we require the lightcurve to initially rise), $0<t_{\rm peak}<300$ (since our data only covers the period up to 200 days) and $s<100$. The parameters will be passed to the function via a tuple.

In [ ]:
def lnprior(theta):
    S_peak, t_peak, delta_1, delta_2, alpha, log_s = theta
    
    #If the prior conditions are met, return 0.0
    if #Prior conditions:
        return 0.0
    
    #Otherwise return -infinity
    else:
        return -np.inf

We will now write a likelihood function that takes the lightcurve parameters inside the tuple `theta`, along with the observed data.

In [ ]:
def lnlike(theta, t, nu, S, S_err):
    S_peak, t_peak, delta_1, delta_2, alpha, log_s = theta
    
    model = ####
    inv_sigma2 = ####
    
    lnlike_val = ####
    return lnlike_val

We then use a function to calculate the marginal probability using the `lnlike()` and `lnprior()` functions from above

In [ ]:
def lnprob(theta, t, nu, S, S_err):
    lp = lnprior(theta)

    if ####:
        return -np.inf
    # otherwise return posterial distribution
    return lp + lnlike(theta, t, nu, S, S_err)

We will now fit the data using the `emcee` package. The function `get_starting_pos()` provided below will set up an array of walker starting positions for given lightcurve parameters. Examine the lightcurve and estimate some reasonable values for these parameters and add them to the function.

In [ ]:
def get_starting_pos(nwalkers, ndim=6):
    # set initial position for all the parameters to fit
    S_peak =
    t_peak = 
    delta_1 = 
    delta_2 = 
    alpha = 
    log_s = 
    
    pos = 
    
    return pos

We write a function called `run_mcmc()` that will load the observed data, take the starting position and then run the emcee Ensemple Sampler. Use a small number of iterations and walkers initially (100/20) to see how long the code takes to run on your laptop. Then increase both parameters to a larger number so that the algorithm takes ~90 seconds to run.

In [ ]:
def run_mcmc(data, niters=, nthreads=1, nwalkers=, ndim=):
    
    
    pos = get_starting_pos(nwalkers, ndim=ndim)
    
    sampler = emcee.EnsembleSampler()
    
    start = timer()
    sampler.run_mcmc()
    end = timer()
    
    print("Computation time: %f s"%(end-start))
    
    return sampler

# Step 5: Making Plots and Evaluate Goodness of Fitting

We now want to inspect our chain to see if our algorithm has converged to a reasonable solution. We extract the chain from the sampler and then write a function to make a figure showing how each walker moves around the parameter space. The figure has 6 subplots (1 for each dimension), iteration number on the x-axis and parameter value on the y-axis.

MCMC algorithms typically use a burn-in phase, where the sampler is moving towards the optimum solution and not yet accurately sampling the parameter space. Add a parameter chain_cut to your function that plots a vertical line at the end of the burn-in.

In [ ]:
sampler = run_mcmc(tdata)
chain = sampler.chain

In [ ]:
def make_chain_plot(chain, chain_cut):
    niters = chain.shape[1]
    ndim = chain.shape[2]

    fig, axes = plt.subplots(ndim,1,sharex=True)
    fig.set_size_inches(7, 20)
    
    param_names = ['$S_{{\\nu,\\rm peak}, 3\.{\\rm GHz}}$', '$t_{{\\rm peak}}$','$\\delta_1$','$\\delta_2$', '$\\alpha$', '$\\log_{10}(s)$']

    for i, (ax,param_name) in enumerate(zip(axes,param_names)):
        #plot the chain for the given parameter
        ax.plot(chain[:,:,i].T,linestyle='-',color='k',alpha=0.3)        
        
        ax.set_ylabel(param_name)
        ax.set_xlim(0,niters)
        
        ax.axvline(chain_cut,c='r',linestyle='--')

chain_cut = ######

make_chain_plot(chain, chain_cut)

Now that we know that our algorithm is converging, and we know how long the burn-in takes we can begin to estimate parameters. The function below will make a **corner** plot from the good part of your chain (using the `corner` package).

In [ ]:
good_chain = chain[:, chain_cut:, :]

In [ ]:
def make_corner_plot(good_chain, savefile='corner.png'):
    param_names = ['$S_{{\\nu,\\rm peak}, 3\.{\\rm GHz}}$', '$t_{{\\rm peak}}$','$\\delta_1$','$\\delta_2$', '$\\alpha$', '$\\log_{10}(s)$']
    ndim = good_chain.shape[2]
    
    fig = corner.corner(good_chain.reshape((-1, ndim)), labels=param_names, quantiles=[0.16, 0.5, 0.84], show_titles=True)
    plt.savefig(savefile)

make_corner_plot(good_chain)

The function below will then extract the median and uncertainty (1 standard deviation) from the chain.

In [ ]:
def get_best_params(chain):
    ndim = chain.shape[2]
    
    chain = chain.reshape((-1, ndim))
    vals = map(lambda v: (v[1], v[2]-v[1], v[1]-v[0]), zip(*np.percentile(chain, [16, 50, 84],axis=0)))
    
    param_names = ['S_peak', 't_peak', 'delta_1', 'delta_2', 'alpha', 'log_s']
    
    param_dict = dict(zip(param_names,vals))
    
    return param_dict
    
    
best_params = get_best_params(good_chain)

Now write a function, `calc_chi2()`, that will calculate the $\chi^2$ for the fit. We will use this later to compare different lightcurve models

In [ ]:
def calc_chi2(best_params, param_names, model, data, nu0=3.0):
    args = []
    for param in param_names:
        val = 
        args.append(val)

    best_fit = model(data['delta_t'], nu0, *args)
    
    chi2 = ######
    
    return chi2

param_names = ['F_peak', 't_peak', 'delta_1', 'delta_2', 'alpha', 'log_s']

chi2_best = calc_chi2(best_params, param_names, smooth_broken_power_law, tdata)
print(chi2_best)

We will now plot our best fit on top of the observational data.

Fill in the function `plot_model()` that takes in a function that calculates the model fit (in this case, our `smooth_broken_power_law` function), the best parameters, an array of values to plot the model for and a matplotlib axis to plot it on.

In [ ]:
def plot_model(model, params, tvals, ax):
    best_fit = ######
    
    ax.plot(tvals,best_fit,marker='',linestyle='-',c='k',linewidth=1.5,zorder=0)
    
    return

plotting_data = scale_data(tdata, best_params['alpha'][0], np.max(best_params['alpha'][1:]))    
    
make_plot(plotting_data, scaled=True, model=smooth_broken_power_law, params=args)

In [ ]:
make_plot(plotting_data, scaled=True, model=smooth_broken_power_law, params=full_args, plot_models=True)

An [F-test](https://en.wikipedia.org/wiki/F-test) is a generalised test that can be used to compare statistical models. In particular, it is useful when comparing two models where one is a restricted form of the other. Write a function calculate_ftest that calculates the F statistic for our two fits and then calculates the corresponding p-value. Hint: We have already imported the [scipy.stats F-distribution model](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.f.html), and we can access the cumulative distribution function using f.cdf().

In [ ]:
def calculate_ftest(chi2_t, p_t, chi2_nt, p_nt, n):
    F = ######
    
    pval = f.cdf(F, p_nt, p_t)
    
    return 1-pval

n = ######
p_t = ######
p_nt = ######

calculate_ftest(chi2_best, p_t, chi2_nt, p_nt, n)

Which model is preferred? With what confidence can we say that we prefer one model over the other?